# Jonathan Halverson
# Wednesday, March 15, 2017
# Active, ranked and weight classes

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('halverson')
%matplotlib inline

In [2]:
ufc = pd.read_csv('data/ufc_dot_com_fighter_data_CLEAN_28Feb2017.csv', header=0)
ufc.head(3)

,Name,Nickname,Active,Record,Age,Height,Weight,Reach,LegReach,OutOf,From,College,Degree,Summary
0,Danny Abbadi,The Assassin,0,NaN,33.0,71.0,185.0,NaN,NaN,NaN,Amman Jordan,NaN,NaN,Striking and kicks
1,Tank Abbott,Tank,0,NaN,51.0,72.0,250.0,NaN,NaN,NaN,"Orange, California USA",NaN,NaN,"Punching power, strength"
2,Shamil Abdurakhimov,Abrek,1,17-4-0,35.0,75.0,235.0,76.0,41.0,Dagestan Russia,Dagestan Russia,NaN,NaN,Wrestling and boxing


In [3]:
ufc[ufc.Active == 1].shape[0]

533

In [4]:
iofile = 'data/fightmetric_fighters_with_corrections_from_UFC_Wikipedia_CLEAN.csv'
fighters = pd.read_csv(iofile, header=0, parse_dates=['Dob'])
fighters.head(3)

,Name,Nickname,Dob,Weight,Height,Reach,LegReach,Stance,Win,Loss,Draw
0,Tom Aaron,NaN,1978-07-13,155.0,NaN,NaN,NaN,NaN,5.0,3.0,0.0
1,Danny Abbadi,The Assassin,1983-07-03,155.0,71.0,NaN,NaN,Orthodox,4.0,6.0,0.0
2,David Abbott,Tank,NaT,265.0,72.0,NaN,NaN,Switch,10.0,14.0,0.0


In [5]:
iofile = 'data/fightmetric_cards/fightmetric_fights_CLEAN_3-6-2017.csv'
fights = pd.read_csv(iofile, header=0, parse_dates=['Date'])
fights.head(3)

,Winner,Outcome,Loser,WeightClass,Method,MethodNotes,Round,Time,Event,Date,Location
0,Germaine de Randamie,def.,Holly Holm,Women's Featherweight,U-DEC,NaN,5,5:00,UFC 208: Holm vs. De Randamie,2017-02-11,"New York, New York, USA"
1,Anderson Silva,def.,Derek Brunson,Middleweight,U-DEC,NaN,3,5:00,UFC 208: Holm vs. De Randamie,2017-02-11,"New York, New York, USA"
2,Jacare Souza,def.,Tim Boetsch,Middleweight,SUB,Kimura,1,3:41,UFC 208: Holm vs. De Randamie,2017-02-11,"New York, New York, USA"


In [6]:
with open('data/ranked_ufc_fighters_1488838405.txt') as f:
     ranked = f.readlines()
ranked = [fighter.strip() for fighter in ranked]

### Is anyone ranked but not in active?

In [7]:
set(ranked) - set(ufc[ufc.Active == 1].Name)

{'Jacare Souza', 'Rogerio Nogueira', 'Tim Johnson'}

In [8]:
ufc[ufc.Name.str.contains('Souza')]

,Name,Nickname,Active,Record,Age,Height,Weight,Reach,LegReach,OutOf,From,College,Degree,Summary
367,Tony DeSouza,NaN,0,NaN,42.0,73.0,170.0,NaN,NaN,NaN,Lima Peru,NaN,NaN,"Wrestling, takedowns, submissions"
1621,Kevin Souza,NaN,0,NaN,32.0,72.0,145.0,74.0,44.0,Florianopolis Brazil,"Salvador, Bahia Brazil",NaN,NaN,Striking
1622,Ronaldo Souza,Jacare,1,"24-4-0, 1NC",37.0,73.0,185.0,72.0,41.0,Rio de Janeiro Brazil,Espiritos Santos Brazil,NaN,NaN,"Submissions, takedowns, strength"


In [9]:
idx = ufc[ufc.Name == 'Ronaldo Souza'].index
ufc = ufc.set_value(idx, 'Name', 'Jacare Souza')
idx = ufc[ufc.Name == 'Timothy Johnson'].index
ufc = ufc.set_value(idx, 'Name', 'Tim Johnson')
idx = ufc[ufc.Name == 'Antonio Rogerio Nogueira'].index
ufc = ufc.set_value(idx, 'Name', 'Rogerio Nogueira')

In [10]:
set(ranked) - set(ufc[ufc.Active == 1].Name)

set()

### Anyone ranked or active but not in the fighter database?

In [11]:
set(ranked) - set(fighters.Name)

set()

In [12]:
set(ufc[ufc.Active == 1].Name) - set(fighters.Name)

{'Andre Soukhamthath',
 'Chris De La Rocha',
 'Cindy Dandois',
 'Cynthia Calvillo',
 'Daniel Spitz',
 'Desmond Green',
 'Dmitriy Sosnovskiy',
 'J.C. Cottrell',
 'Jim Wallhead',
 'Joshua Stansbury',
 'Marco Polo Reyes',
 'Marcos Rogerio De Lima',
 'Mike De La Torre',
 'Rich Walsh',
 'Saparbek Safarov'}

In [13]:
from fuzzywuzzy import process

for fighter in set(ufc[ufc.Active == 1].Name) - set(fighters.Name):
     best_match, score = process.extractOne(query=fighter, choices=fighters.Name)
     if score >= 87:
          idx = ufc[ufc.Name == fighter].index
          ufc = ufc.set_value(idx, 'Name', best_match)
          print fighter, '-->', best_match, score

Chris De La Rocha --> Chris de la Rocha 100
Dmitriy Sosnovskiy --> Dmitry Sosnovskiy 97
Jim Wallhead --> Jimmy Wallhead 92
Joshua Stansbury --> Josh Stansbury 93
J.C. Cottrell --> JC Cottrell 92
Marcos Rogerio De Lima --> Marcos Rogerio de Lima 100
Saparbek Safarov --> Saparbeg Safarov 94
Rich Walsh --> Richard Walsh 87
Marco Polo Reyes --> Polo Reyes 90
Mike De La Torre --> Mike de la Torre 100


In [14]:
set(ufc[ufc.Active == 1].Name) - set(fighters.Name)

{'Andre Soukhamthath',
 'Cindy Dandois',
 'Cynthia Calvillo',
 'Daniel Spitz',
 'Desmond Green'}

We see only five fighters were not in the database. Soukhamthath is 0-1, Dandois is a recent signing, Calvillo is 1-0, Spitz is 0-1, and Green is 0-0.

### Now that data is consistent, let's try to find the best weight class for each fighter:

In [15]:
fights.WeightClass.value_counts()

Lightweight              827
Welterweight             789
Middleweight             621
Heavyweight              426
Light Heavyweight        402
Featherweight            320
Bantamweight             261
Flyweight                131
Open Weight              125
Women's Bantamweight      76
Women's Strawweight       68
Catch Weight              19
Women's Featherweight      1
Women's Flyweight          1
Super Heavyweight          1
Name: WeightClass, dtype: int64

In [16]:
f = 'Jessica Andrade'
wins = fights[fights.Winner == f][['Winner', 'WeightClass']]
loses = fights[fights.Loser == f][['Loser', 'WeightClass']]
loses.columns = ['Winner', 'WeightClass']
wins.append(loses).WeightClass.value_counts().sort_values(ascending=False).index[0]

"Women's Bantamweight"

In [17]:
win_lose = fights.Winner.append(fights.Loser).unique()
fighter_weightclass = []
for fighter in win_lose:
     wins = fights[fights.Winner == fighter][['Winner', 'WeightClass']]
     loses = fights[fights.Loser == fighter][['Loser', 'WeightClass']]
     loses.columns = ['Winner', 'WeightClass']
     weightclass = wins.append(loses).WeightClass.value_counts().sort_values(ascending=False).index[0]
     fighter_weightclass.append((fighter, weightclass))

In [18]:
majority = pd.DataFrame(fighter_weightclass)
majority.columns = ['Name', 'WeightClass']
majority.WeightClass.value_counts()

Welterweight            315
Lightweight             310
Middleweight            222
Heavyweight             179
Light Heavyweight       141
Featherweight           130
Bantamweight            105
Open Weight              96
Flyweight                55
Women's Strawweight      45
Women's Bantamweight     37
Catch Weight              6
Name: WeightClass, dtype: int64

In [19]:
majority.shape[0]

1641

In [20]:
majority = majority.merge(ufc[['Name', 'Active']], on='Name', how='left')

In [21]:
majority[majority.WeightClass == 'Open Weight']

,Name,WeightClass,Active
842,Mark Coleman,Open Weight,0.0
950,Ken Shamrock,Open Weight,0.0
991,Amaury Bitetti,Open Weight,NaN
1000,Kenichi Yamamoto,Open Weight,0.0
1002,Katsuhisa Fujii,Open Weight,NaN
1014,Gary Goodridge,Open Weight,0.0
1020,David Abbott,Open Weight,NaN
1021,Jerry Bohlander,Open Weight,0.0
1026,Dave Beneteau,Open Weight,0.0
1031,Joe Moreira,Open Weight,NaN


In [22]:
idx = majority[majority.Name == 'Ken Shamrock'].index
majority = majority.set_value(idx, 'WeightClass', 'Light Heavyweight')

In [23]:
majority[majority.WeightClass == 'Catch Weight']

,Name,WeightClass,Active
39,Augusto Mendes,Catch Weight,1.0
180,Cris Cyborg,Catch Weight,1.0
1110,Lina Lansberg,Catch Weight,1.0
1135,Darrell Horcher,Catch Weight,1.0
1200,Alexis Dufresne,Catch Weight,0.0
1471,Joe Jordan,Catch Weight,0.0


In [24]:
idx = majority[majority.Name == 'Augusto Mendes'].index
majority = majority.set_value(idx, 'WeightClass', 'Bantamweight')
idx = majority[majority.Name == 'Darrell Horcher'].index
majority = majority.set_value(idx, 'WeightClass', 'Lightweight')
idx = majority[majority.Name == 'Alexis Dufresne'].index
majority = majority.set_value(idx, 'WeightClass', "Women's Bantamweight")
idx = majority[majority.Name == 'Joe Jordan'].index
majority = majority.set_value(idx, 'WeightClass', 'Lightweight')

In [25]:
f = 'Ken Shamrock'
fights[(fights.Winner == f) | (fights.Loser == f)]

,Winner,Outcome,Loser,WeightClass,Method,MethodNotes,Round,Time,Event,Date,Location
3425,Tito Ortiz,def.,Ken Shamrock,Light Heavyweight,KO/TKO,Punches,1,2:23,Ortiz vs Shamrock 3: The Final Chapter,2006-10-10,"Hollywood, Florida, USA"
3463,Tito Ortiz,def.,Ken Shamrock,Light Heavyweight,KO/TKO,Elbows,1,1:18,UFC 61: Bitter Rivals,2006-07-08,"Las Vegas, Nevada, USA"
3602,Rich Franklin,def.,Ken Shamrock,Light Heavyweight,KO/TKO,Punches,1,2:42,The Ultimate Fighter 1 Finale,2005-04-09,"Las Vegas, Nevada, USA"
3635,Ken Shamrock,def.,Kimo Leopoldo,Heavyweight,KO/TKO,Knee,1,1:26,UFC 48: Payback,2004-06-19,"Las Vegas, Nevada, USA"
3700,Tito Ortiz,def.,Ken Shamrock,Light Heavyweight,KO/TKO,NaN,3,5:00,UFC 40: Vendetta,2002-11-22,"Las Vegas, Nevada, USA"
3949,Ken Shamrock,def.,Brian Johnston,Open Weight,SUB,Forearm Choke,1,5:48,UFC - Ultimate Ultimate '96,1996-12-07,"Birmingham, Alabama, USA"
3973,Dan Severn,def.,Ken Shamrock,Open Weight,S-DEC,NaN,3,3:00,UFC 9: Motor City Madness,1996-05-17,"Detroit, Michigan, USA"
3981,Ken Shamrock,def.,Kimo Leopoldo,Open Weight,SUB,Kneebar,1,4:24,UFC 8: David vs Goliath,1996-02-16,"San Juan, Puerto Rico"
3999,Ken Shamrock,draw,Oleg Taktarov,Open Weight,Other,NaN,2,3:00,UFC 7: The Brawl in Buffalo,1995-09-08,"Buffalo, New York, USA"
4010,Ken Shamrock,def.,Dan Severn,Open Weight,SUB,Guillotine Choke,1,2:14,UFC 6: Clash of the Titans,1995-07-14,"Casper, Wyoming, USA"


Now check for mistakes between the rankings by weight class and the assigned weight class:

In [26]:
wc = ['Flyweight', 'Bantamweight', 'Featherweight', 'Lightweight', 'Welterweight',
     'Middleweight', 'Light Heavyweight', 'Heavyweight', "Women's Strawweight", "Women's Bantamweight"]
ranked_weightclass = []
for i, w in enumerate(wc):
     for j in range(16):
          ranked_weightclass.append((ranked[i * 16 + j], w))

In [27]:
by_rank = pd.DataFrame(ranked_weightclass)
by_rank.columns = ['Name', 'WeightClass']
z = majority.merge(by_rank, on='Name', how='inner', suffixes=('_majority', '_by_rank'))
z[z.WeightClass_majority != z.WeightClass_by_rank]

,Name,WeightClass_majority,Active,WeightClass_by_rank
4,Dustin Poirier,Featherweight,1.0,Lightweight
10,Jessica Andrade,Women's Bantamweight,1.0,Women's Strawweight
13,Jorge Masvidal,Lightweight,1.0,Welterweight
31,Donald Cerrone,Lightweight,1.0,Welterweight
33,Kelvin Gastelum,Welterweight,1.0,Middleweight
44,Robert Whittaker,Welterweight,1.0,Middleweight
56,Conor McGregor,Featherweight,1.0,Lightweight
61,Frankie Edgar,Lightweight,1.0,Featherweight
73,John Lineker,Flyweight,1.0,Bantamweight
74,Renan Barao,Bantamweight,1.0,Featherweight


In [28]:
idx = majority[majority.Name == 'Anthony Johnson'].index
majority = majority.set_value(idx, 'WeightClass', 'Light Heavyweight')

### Each fighter has been assigned to a single weight class so write to file:

In [29]:
majority.columns = ['Name', 'WeightClassMajority', 'Active']
majority.to_csv('data/weight_class_majority.csv', index=False)